# Imports

In [ ]:
from aoc import *
import re
import os
import itertools
import math
from aocd.models import Puzzle as AOCDPuzzle


def Puzzle(day, year=2024):
    return AOCDPuzzle(year=year, day=day)


%load_ext line_profiler

# Day 1 - Historian Hysteria

In [ ]:
p = Puzzle(1)
p

In [ ]:
lines = p.input_data.splitlines()
pairs = [tuple(int(i) for i in line.split("  ")) for line in lines]
lists = [sorted([tpl[i] for tpl in pairs]) for i in range(2)]
p.answer_a = sum(abs(a - b) for a, b in zip(*lists))

In [ ]:
from collections import Counter

counts = Counter(lists[1])
p.answer_b = sum(val * counts[val] for val in lists[0])

# Day 2 - Red-Nosed Reports

In [ ]:
p = Puzzle(year=2024, day=2)
p

In [ ]:
reports = [vector(line) for line in p.input_data.splitlines()]


def is_safe(report):
    diffs = [(b - a) for a, b in itertools.pairwise(report)]
    return all(1 <= abs(d) <= 3 for d in diffs) and (
        all(d > 0 for d in diffs) or all(d < 0 for d in diffs)
    )


def signum(i):
    return -1 if i < 0 else 1 if i > 0 else 0


def dampen(report):
    return is_safe(report) or (
        any(is_safe(report[:i] + report[i + 1 :]) for i in range(len(report)))
    )


p.answer_a = len([r for r in reports if is_safe(r)])
p.answer_b = len([r for r in reports if dampen(r)])

# Day 3 - Null It Over

In [ ]:
p = Puzzle(day=3)
p

In [ ]:
p.answer_a = sum(
    int(x) * int(y) for x, y in re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", p.input_data)
)

In [ ]:
def null_it_over_p2(input: str):
    instrux = re.findall(r"mul\((\d{1,3}),(\d{1,3})\)|(do|don't)\(\)", input)
    enabled = True
    sum = 0
    for x, y, mode in instrux:
        if mode == "do":
            enabled = True
        elif mode == "don't":
            enabled = False
        elif enabled:
            sum += int(x) * int(y)
    return sum


assert 48 == null_it_over_p2(
    """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5)))"""
)

p.answer_b = null_it_over_p2(p.input_data)

# Day 4 - Ceres Search

In [ ]:
p = Puzzle(4)
p

In [ ]:
SEARCH_DIR = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]


def word_search(grid, p, dir, word) -> bool:
    for i in range(len(word)):
        q = (p[0] + dir[0] * i, p[1] + dir[1] * i)
        if grid.get(q) != word[i]:
            return False
    return True


def find_words(input: str, word="XMAS") -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    xes = [p for p, c in grid.items() if c == "X"]
    answer = 0
    for x in xes:
        answer += sum(word_search(grid, x, dir, word) for dir in SEARCH_DIR)
    return answer


def is_x_mas(grid, a) -> bool:
    # Only diagonals!
    neighbors = [(-1, -1), (1, 1), (-1, 1), (1, -1)]
    letters = "".join(grid.get((a[0] + n[0], a[1] + n[1]), "") for n in neighbors)
    return len(letters) == 4 and letters in ("MSMS", "SMSM", "SMMS", "MSSM")


def find_x_mas(input: str) -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    a_s = [p for p, c in grid.items() if c == "A"]
    return len([a for a in a_s if is_x_mas(grid, a)])


assert 18 == find_words(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_a = find_words(p.input_data)

assert 9 == find_x_mas(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_b = find_x_mas(p.input_data)

# Day 5

In [ ]:
p = Puzzle(day=5)
p

In [ ]:
from collections import defaultdict
import functools


def sort_update(update, rules):
    def page_cmp(a, b):
        r1, r2 = rules[a], rules[b]
        if b in r1:
            return -1
        if a in r2:
            return 1
        return 0

    return tuple(sorted(update, key=functools.cmp_to_key(page_cmp)))


def update_is_sorted(update, rules):
    return update == sort_update(update, rules)


def parse_rules(input: str):
    rule_text, updates = input.split("\n\n")
    rules = defaultdict(set)
    for before, after in [
        vector(line.replace("|", ",")) for line in rule_text.splitlines()
    ]:
        rules[before].add(after)
    updates = [vector(line) for line in updates.splitlines()]
    return rules, updates


def print_jobs(input: str) -> int:
    rules, updates = parse_rules(input)
    total = 0
    for update in updates:
        is_sorted = update_is_sorted(update, rules)
        if is_sorted:
            mid = update[len(update) // 2]
            total += mid
    return total


def sort_jobs(input: str) -> int:
    rules, updates = parse_rules(input)
    total = 0
    for update in updates:
        new_rules = dict((k, v & set(update)) for k, v in rules.items() if k in update)
        supdate = sort_update(update, new_rules)
        if update != supdate:
            mid = supdate[len(supdate) // 2]
            total += mid

    return total


assert 143 == print_jobs(p.examples[0].input_data)

In [ ]:
p.answer_a = print_jobs(p.input_data)

In [ ]:
p.answer_b = sort_jobs(p.input_data)

# Day 6 - Guard Gallivant

In [ ]:
p = Puzzle(day=6)
p

In [ ]:
TURNS = {(-1, 0): (0, 1), (0, 1): (1, 0), (1, 0): (0, -1), (0, -1): (-1, 0)}


def parse_grid(input: str) -> dict:
    return dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )


def find_loop(grid: dict, pos: tuple, heading: tuple) -> bool:
    seen = set()
    save = pos
    try:
        grid[save] = "#"
        pos = (pos[0] - heading[0], pos[1] - heading[1])
        while True:
            if (pos, heading) in seen:
                return True
            seen.add((pos, heading))
            p = (pos[0] + heading[0], pos[1] + heading[1])
            c = grid.get(p)
            if c is None:
                return False
            if c == "#":
                heading = TURNS[heading]
                continue
            pos = p
        return False
    finally:
        grid[save] = "."


def guard_path(grid: dict) -> bool:
    guard = next(p for p, c in grid.items() if c == "^")
    heading = (-1, 0)
    path = set()
    seen = set()
    while True:
        if (guard, heading) in seen:
            raise ValueError("Loop detected")
        path.add(guard)
        seen.add((guard, heading))
        p = (guard[0] + heading[0], guard[1] + heading[1])
        c = grid.get(p)
        if c is None:
            break
        if c == "#":
            heading = TURNS[heading]
            continue
        guard = p
    return path


def guard_gallivant(input: str) -> int:
    return len(guard_path(parse_grid(input)))


def find_loops(input: str) -> int:
    grid = parse_grid(input)
    loops = 0
    pos = next(p for p, c in grid.items() if c == "^")
    heading = (-1, 0)
    seen = set()

    while True:
        seen.add(pos)
        p = (pos[0] + heading[0], pos[1] + heading[1])
        if p not in grid:
            break
        if grid.get(p) == "#":
            heading = TURNS[heading]
            p = (pos[0] + heading[0], pos[1] + heading[1])
        if p not in seen and find_loop(grid, p, heading):
            loops += 1
        pos = p

    return loops


assert (
    guard_gallivant(
        """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
    )
    == 41
)

assert (
    find_loops(
        """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
    )
    == 6
)

In [ ]:
p.answer_a = guard_gallivant(p.input_data)

In [ ]:
p.answer_b = find_loops(p.input_data)

# Day 7 - Bridge Repair

In [ ]:
p = Puzzle(day=7)
p

In [ ]:
import operator
import itertools

pow10_table = [10**i for i in range(1, 10)]


def concat(n1, n2):
    for k in pow10_table:
        if n2 - k < 0:
            return n1 * k + n2
    assert False, f"{n1} {n2}"


# First pass: brute force solution - ~15s on part 2
def solve_equation(answer: int, terms: list[int], operators) -> int:
    for ops in itertools.product(operators, repeat=len(terms) - 1):
        total = terms[0]
        for val, op in zip(terms[1:], ops):
            total = op(total, val)
            if total > answer:
                break
        else:
            if total == answer:
                return ops
    return None


# RTL recursive solver based on solutions seen on AoC Reddit.
def solver(answer: int, terms: list[int], use_concat=False) -> int:
    if len(terms) == 1:
        return terms[0] == answer
    if answer <= 0:
        return False
    head, tail = terms[:-1], terms[-1]
    return (
        # Addition
        (solver(answer - tail, head, use_concat) if answer > tail else False)
        # Multiplication
        or (solver(answer // tail, head, use_concat) if answer % tail == 0 else False)
        # Concatenation
        or (
            solver(answer // 10 ** len(str(tail)), head, use_concat)
            if (use_concat and str(answer).endswith(str(tail)))
            else False
        )
    )


def bridge_repair(input: str, use_concat=False) -> int:
    total = 0
    # operators = (operator.add, operator.mul) + (concat,) * use_concat
    for line in input.splitlines():
        answer, *terms = [int(i) for i in re.findall(r"\d+", line)]
        if solver(answer, terms, use_concat):
            total += answer
    return total


assert 3749 == bridge_repair(p.examples[0].input_data)

assert 11387 == bridge_repair(p.examples[0].input_data, True)

In [ ]:
p.answer_a = bridge_repair(p.input_data)

In [ ]:
p.answer_b = bridge_repair(p.input_data, True)

# Day 8: Resonant Collinearity

In [ ]:
p = Puzzle(day=8)
p

In [ ]:
def parse_antennas(input: str) -> dict:
    graph = parse_grid(input)
    antennas = defaultdict(list)
    for p, c in graph.items():
        if c.isdigit() or c.isalpha():
            antennas[c].append(p)
    return graph, antennas


def antinodes(input: str) -> int:
    graph, antennas = parse_antennas(input)
    antinodes = set()
    for nodes in antennas.values():
        for p, q in itertools.combinations(nodes, 2):
            dy, dx = q[0] - p[0], q[1] - p[1]
            for antinode in ((p[0] - dy, p[1] - dx), (q[0] + dy, q[1] + dx)):
                if antinode in graph:
                    antinodes.add(antinode)

    return len(antinodes)


def antinode_harmonics(input: str) -> int:
    graph, antennas = parse_antennas(input)
    height, width = max(graph.keys())
    antinodes = set()
    for nodes in antennas.values():
        for p, q in itertools.combinations(nodes, 2):
            dy, dx = q[0] - p[0], q[1] - p[1]
            for i in range(max(height, width) + 1):
                nodes = [
                    n
                    for n in (
                        (p[0] - dy * i, p[1] - dx * i),
                        (q[0] + dy * i, q[1] + dx * i),
                    )
                    if n in graph
                ]
                if not nodes:
                    break
                antinodes.update(nodes)

    return len(antinodes)


assert 14 == antinodes(p.examples[0].input_data)
assert 34 == antinode_harmonics(p.examples[0].input_data)

In [ ]:
p.answer_a = antinodes(p.input_data)

In [ ]:
p.answer_b = antinode_harmonics(p.input_data)

# Day 9 - Disk Fragmenter

In [ ]:
p = Puzzle(day=9)
p

In [ ]:
import heapq


def build_filesystem(input: str):
    used = []
    free = []
    offset = 0
    for i, c in enumerate(input):
        num_blocks = int(c)
        if not num_blocks:
            continue
        if i % 2 == 0:
            # Files are stored as (-end, -start, fileno)
            used.append((-offset - num_blocks, -offset, i // 2))
        else:
            # Free space represnted as (start, end, None)
            free.append((offset, offset + num_blocks, None))
        offset += num_blocks
    heapq.heapify(used)
    heapq.heapify(free)
    return used, free


def defrag(input: str) -> int:
    used, free = build_filesystem(input)
    while free and free[0][0] < -used[0][1]:  # min(free)[0] < -(min(used)[0]):
        end, start, fileno = heapq.heappop(used)
        fstart, fend, _ = heapq.heappop(free)
        file_size = start - end  # negative numbers
        free_blocks = fend - fstart
        moved_blocks = min(file_size, free_blocks)
        heapq.heappush(used, (-fstart - moved_blocks, -fstart, fileno))
        if moved_blocks < file_size:
            heapq.heappush(used, (end + moved_blocks, start, fileno))
        fstart += moved_blocks
        free_blocks -= moved_blocks
        if free_blocks > 0:
            heapq.heappush(free, (fstart, fend, None))
    total = 0
    for end, start, fileno in used:
        total += sum(fileno * n for n in range(-start, -end))
    return total


def defrag_wholefile(input: str) -> int:
    used, free = build_filesystem(input)
    free_blocks = sorted(free)
    result = dict()
    for end, start, fileno in sorted(used, key=lambda x: x[2], reverse=True):
        file_size = start - end
        free_block = None
        for i, h in enumerate(free_blocks):
            if h[0] > -start:
                break
            if h[1] - h[0] >= file_size:
                free_block = (i, h)
                break
        if free_block is None:  # No room
            result[fileno] = (-start, -end)
            continue
        i, (fstart, fend, _) = free_block
        assert fend - fstart >= file_size
        result[fileno] = (fstart, fstart + file_size)
        fstart += file_size
        if fstart < fend:
            free_blocks[i] = (fstart, fend, None)
        else:
            free_blocks.pop(i)
    total = 0
    for fileno, (start, end) in result.items():
        total += sum(fileno * n for n in range(start, end))
    return total


assert 1928 == defrag("2333133121414131402")
assert 2858 == defrag_wholefile("2333133121414131402")

In [ ]:
# p.answer_a =
print(p.answers)
defrag(p.input_data.strip())

In [ ]:
# p.answer_b =
defrag_wholefile(p.input_data.strip())

# Day 10 - Hoof It

In [ ]:
p = Puzzle(day=10)
p

In [ ]:
def moves(grid, p):
    start = int(grid[p])
    for q in neighbors4(p):
        if q in grid and grid[q].isdigit() and int(grid[q]) == start + 1:
            yield q


def score_and_rating(start, goals, moves):
    """Count number of distinct paths from start to any goal"""
    paths = 0
    reached = set()
    todo = [(start,)]
    while todo:
        path = todo.pop()
        p = path[-1]
        if p in goals:
            reached.add(p)
            paths += 1
            continue
        for q in moves(p):
            if q not in path:
                todo.append(path + (q,))
    return len(reached), paths


def hoof_it(input: str) -> int:
    grid = parse_grid(input)
    trailheads = [p for p, c in grid.items() if c == "0"]
    nines = [p for p, c in grid.items() if c == "9"]

    moves_func = lambda p: moves(grid, p)
    scores_and_ratings = [score_and_rating(t, nines, moves_func) for t in trailheads]
    scores = sum(s for s, r in scores_and_ratings)
    ratings = sum(r for s, r in scores_and_ratings)
    return scores, ratings


assert 36, 81 == hoof_it(
    """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""
)

In [ ]:
p.answers = hoof_it(p.input_data)

# Day 11 - Plutonian Pebbles

In [ ]:
p = Puzzle(day=11)
p

In [ ]:
def pebble_rules(p):
    if p == 0:
        yield 1
    else:
        pstr = str(p)
        if len(pstr) % 2 == 0:
            yield int(pstr[: len(pstr) // 2])
            yield int(pstr[len(pstr) // 2 :])
        else:
            yield 2024 * p


def plutonian_pebbles(input: str, blinks: int) -> int:
    pebbles = vector(input)
    counts = Counter(pebbles)
    for _ in range(blinks):
        newcounts = Counter()
        for p, count in counts.items():
            for q in pebble_rules(p):
                newcounts[q] += count
        counts = newcounts
    return sum(counts.values())


assert 55312 == plutonian_pebbles("125 17", 25)

In [ ]:
p.answer_a = plutonian_pebbles(p.input_data, 25)

In [ ]:
p.answer_b = plutonian_pebbles(p.input_data, 75)

# Day 12

In [ ]:
p = Puzzle(day=12)
p

In [ ]:
p.view()

In [ ]:
def perimeter(graph, region) -> int:
    answer = 0
    for p in region:
        c = graph[p]
        answer += 4 - sum(1 for q in neighbors4(p) if graph.get(q) == c)
    return answer


def count_sides(region) -> int:
    sides = defaultdict(set)
    outside = set(p for p in region if not all(n in region for n in neighbors4(p)))
    for y, x in outside:
        if not (y - 1, x) in region:
            sides[("u", y)].add(x)
        if not (y + 1, x) in region:
            sides[("d", y)].add(x)
        if not (y, x - 1) in region:
            sides[("l", x)].add(y)
        if not (y, x + 1) in region:
            sides[("r", x)].add(y)
    num_sides = 0
    for _, points in sides.items():
        num_sides += sum(1 for p in points if p - 1 not in points)
    return num_sides


def find_regions(grid: dict) -> list:
    seen = set()
    regions = []
    for p, c in grid.items():
        if p in seen:
            continue
        region = set([p])
        current = [p]
        while current:
            for n in neighbors4(current.pop()):
                x = grid.get(n)
                if x == c and n not in region:
                    region.add(n)
                    current.append(n)
        # print(c, p, len(region), perimeter(grid, region))
        seen.update(region)
        regions.append(region)
    assert all(p in seen for r in regions for p in r)
    return regions


def garden_groups(input: str) -> int:
    grid = parse_grid(input)
    regions = find_regions(grid)
    total = sum(len(r) * perimeter(grid, r) for r in regions)
    bulk = sum(len(r) * count_sides(r) for r in regions)
    return total, bulk


assert 772, 436 == garden_groups(
    """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""
)

assert 1930, 1206 == garden_groups(
    """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""
)

assert 692, 236 == garden_groups(
    """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""
)

In [ ]:
p.answers = garden_groups(p.input_data)